# INtersection

In [1]:
import json
import re
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

import glob

def lire_fichier (chemin):
    with open(chemin) as json_data: 
        dist =json.load(json_data)
        
        return dist
    
def nom_repertoire(chemin):
    for mot in glob.glob(chemin): 
        noms_rep = re.split("/", chemin)
        noms_repo = re.split("_",noms_rep[5])
        noms_repo = re.split("_",noms_repo[-1])
        noms_repo ="".join(noms_repo)
#        print("NOM FICHIER",noms_fichiers)

        return noms_repo

def nom_mod(path):
    for mot in glob.glob(path): 
        noms_mod = re.split("/", path)
        noms_mods = re.split("_",noms_mod[6])
        nm = re.split("_",noms_mods[-2])
        nmod ="".join(nm)
        return nmod
    

def diagramme_venn(liste_en_pp, liste_en_ocr):
    venn2([set(liste_en_pp), set(liste_en_ocr)],set_labels = ('En PP', 'EN OCR'))
    
    
     
def stocker(nom_fichier):
    plt.savefig(nom_fichier)
    plt.clf()
    
    return nom_fichier
    



        
path_corpora = "./corpora/corpus_eval/*/*/*"
# dans "corpora" un subcorpus = toutes les versions 'un texte'

liste_EN_ocr=[]
liste_EN_pp=[]
#for subcorpus in sorted(glob.glob("%s/*"%path_corpora)):
for subcorpus in sorted(glob.glob(path_corpora)):
#    print(subcorpus)
    for path in sorted(glob.glob("%s/*.json"%subcorpus)):
#        print("subsubcorpus",path )
    
        texte = lire_fichier(path)
    #    print("************",subcorpus)
    #    print(texte)
        nomrep=nom_repertoire(path)
    #    print(nomrep)
        
        if nomrep == "PP" :        
            liste_EN_pp.append(path)
    #        liste_texte_pp.append(subcorpus)
            liste_EN_pp.append(texte)
              
        elif nomrep == "MOD":
            liste_EN_ocr.append(path)
    #         liste_texte_ocr.append(subcorpus)
            liste_EN_ocr.append(texte)
    
    
liste_ocr_verif=[] 
liste_pp_verif=[] 
i=0      
while i <len(liste_EN_ocr) :
    nom_mod_ocr=nom_mod(liste_EN_ocr[i])
    nom_mod_pp=nom_mod(liste_EN_pp[i])
    print(nom_mod_ocr)
    liste_ocr_verif.append(nom_mod_ocr)
    liste_pp_verif.append(nom_mod_pp)
    i=i+2
print(liste_ocr_verif)
print(liste_pp_verif)

result=True
for j in range(0,len(liste_ocr_verif)):
    if liste_ocr_verif[j]!=liste_pp_verif[j]:
        result = False
k=0
if result != False:
    while k<len(liste_EN_pp) :
        
        diagramme_venn(liste_EN_pp[k+1],liste_EN_ocr[k+1])
        stocker("%s_intersection.png"%liste_EN_ocr[k])
        k=k+2
       
    
else:
    print("NOT OK")
 



          


    
    
    

lg
md
sm
lg
md
sm
['lg', 'md', 'sm', 'lg', 'md', 'sm']
['lg', 'md', 'sm', 'lg', 'md', 'sm']


<Figure size 432x288 with 0 Axes>

# Evaluation

In [2]:
import re
import glob
import spacy
import json
import sklearn
import os

from sklearn.neighbors import DistanceMetric
from sklearn.feature_extraction.text import CountVectorizer


def lire_fichier_txt (chemin):
    f = open(chemin , encoding = 'utf−8')
    chaine = f.read ()
    f.close ()
    return chaine

def liste_resultats(texte, nlp=spacy.load("fr_core_news_sm")):
    doc = nlp(texte)
    list_resultats =[]
    for ent in doc.ents:
        if ent.label_=="LOC":
            list_resultats.append(ent.text)
    return (list_resultats)


def stocker( chemin, contenu):

    w =open(chemin, "w")
    w.write(json.dumps(contenu , indent = 2))
    w.close()
    print(chemin)
    


def get_distances(texte1, texte2, N=1, liste_name =["jaccard", "braycurtis","dice", "cosinus"] ):
    dico = {}
    for metric_name in liste_name :
        dico[metric_name] = []
        liste_resultat_dist2 = []
        for n_max in range(1, N+1):###range([min, default = 0], max, [step, default = 1]) 
            V = CountVectorizer(ngram_range=(1,n_max ), analyzer='char') 
            X = V.fit_transform([texte1, texte2]).toarray()
            if metric_name!= "cosinus" :  
                dist = DistanceMetric.get_metric(metric_name)     
                distance_tab1=dist.pairwise(X)
                liste_resultat_dist2.append(distance_tab1[0][1])
            else: 
                distance_tab1=sklearn.metrics.pairwise.cosine_distances(X) 
                liste_resultat_dist2.append(distance_tab1[0][1])
            dico[metric_name] = liste_resultat_dist2
    return dico


## MAIN

for subcorpus in glob.glob('./corpora/corpus_eval/*/*'):
    outputs = glob.glob(f"{subcorpus}/*/*")
    dico_out = {}
    dist_txt = {}
    for file_type in ["txt", "json"]:
        chemins = [x for x in outputs if len(re.findall(f"{file_type}$",x))>0]
        liste_compare = []
        for path_file in chemins:
            filename = re.split("/", path_file)[-1]
            elems = re.split("_|\\.", filename)
            print(elems)
            auteur,titre, version, modele = elems[0], elems[1],elems[2], elems[-3]
            if file_type =="txt":
                liste_compare.append([version, lire_fichier_txt(path_file)])
            else:
                liste_compare.append([modele, lire_fichier_txt(path_file)])
        for c in liste_compare:
            print(file_type, c[0])
        dico_out[file_type] = {}
        for ID1 in range(len(liste_compare)):
            version1 = liste_compare[ID1][0]
            for ID2 in range(ID1+1, len(liste_compare)):
                version2 = liste_compare[ID2][0]
                dico_dist = get_distances(liste_compare[ID1][1], liste_compare[ID2][1], N=5)
                paire = "%s--%s"%(version1, version2)
                dico_out[file_type][paire] = dico_dist
    
    
       
    stocker("%s_%s_distances.json"%(subcorpus,titre), dico_out)
  


['AIMARD', 'les-trappeurs', 'PP', 'txt']
['AIMARD', 'les-trappeurs', 'Kraken-base', 'txt']
txt PP
txt Kraken-base
['AIMARD', 'les-trappeurs', 'PP', 'txt', 'md', 'spacy', 'json']
['AIMARD', 'les-trappeurs', 'PP', 'txt', 'sm', 'spacy', 'json']
['AIMARD', 'les-trappeurs', 'PP', 'txt', 'lg', 'spacy', 'json']
['AIMARD', 'les-trappeurs', 'Kraken-base', 'txt', 'sm', 'spacy', 'json']
['AIMARD', 'les-trappeurs', 'Kraken-base', 'txt', 'md', 'spacy', 'json']
['AIMARD', 'les-trappeurs', 'Kraken-base', 'txt', 'lg', 'spacy', 'json']
json md
json sm
json lg
json sm
json md
json lg
./corpora/corpus_eval/AIMARD_TRAPPEURS/AIMARD-TRAPPEURS_kraken-base_les-trappeurs_distances.json
['AIMARD', 'les-trappeurs', 'PP', 'txt']
['AIMARD', 'les-trappeurs', 'TesseractFra-PNG', 'txt']
txt PP
txt TesseractFra-PNG
['AIMARD', 'les-trappeurs', 'PP', 'txt', 'md', 'spacy', 'json']
['AIMARD', 'les-trappeurs', 'PP', 'txt', 'sm', 'spacy', 'json']
['AIMARD', 'les-trappeurs', 'PP', 'txt', 'lg', 'spacy', 'json']
['AIMARD', 'le